 <a id='section1'>FIRST: </a>Let me do a practise attempt (from intro to data-science in DAZ repo)

In [1]:
import os
from collections import Counter
# import matplotlib.pyplot as plt
import pandas as pd
from dotenv import load_dotenv
from googleapiclient.discovery import build
from textblob import TextBlob

load_dotenv()

youtube_api_key = os.getenv("YOUTUBE_API_KEY")
youtube = build("youtube", "v3", developerKey=youtube_api_key)
request = youtube.videos().list(
    part="snippet,statistics",
    chart="mostPopular",
    regionCode="US",
    maxResults=10,
    videoCategoryId=0,
)
response = request.execute()

# print(response)
# printed to test and is working :)!

video_data = []

for item in response["items"]:
    video_data.append(
        {
            "Title": item["snippet"]["title"],
            "Views": int(item["statistics"]["viewCount"]),
            "Likes": int(item["statistics"].get("likeCount", 0)),
            "Comments": int(item["statistics"].get("commentCount", 0)),
            "CategoryId": item["snippet"]["categoryId"],
        }
    )

video_df = pd.DataFrame(video_data)
print(video_df.head())

                                               Title     Views    Likes  \
0                        Minecraft LIVE - March 2025   3783353   203026   
1         I Survived The 5 Deadliest Places On Earth  28184135  1212778   
2                                      Ye - BULLY V1   2663861   236342   
3           I Hit The World's Fastest Baseball Pitch   1392932    42877   
4  Stray Kids "Burnin’ Tires (Changbin & I.N)" Video   4786099   349706   

   Comments CategoryId  
0      6783         20  
1     22407         24  
2     30253         10  
3      1382         17  
4     21054         10  


Find the category Id of the type of video I want to train for.

In [13]:
url = "https://www.youtube.com/watch?v=MVGJeGWZoy4&t=11s"
video_split = url.split("watch?v=")[-1]
print(video_split)

MVGJeGWZoy4&t=11s


In [ ]:
# Getting rid of timestamping

if "&t=" in video_split:
    video_id = video_split.split("&t=")[0]

else:
    video_id = video_split

video_id

    

'MVGJeGWZoy4'

In [ ]:
# -*- coding: utf-8 -*-

# Sample Python code for youtube.videoCategories.list
# See instructions for running these code samples locally:
# https://developers.google.com/explorer-help/code-samples#python

import os

import googleapiclient.discovery

def main():

    request_category = youtube.videos().list(
        part="snippet",  # We want to fetch the 'snippet' part to get category info
        id=video_id
    )
    response = request_category.execute()

    print(response)

if __name__ == "__main__":
    main()

{'kind': 'youtube#videoListResponse', 'etag': '9VDRg_BWPb73_wPlzTAWqnx0Pec', 'items': [{'kind': 'youtube#video', 'etag': 'VZIgPDeZljSJvB9Ae3kCwX4YH2g', 'id': 'MVGJeGWZoy4', 'snippet': {'publishedAt': '2023-10-19T01:00:14Z', 'channelId': 'UC8PICQUP0a_HsrA9S4IIgWw', 'title': "Jordan Peterson: Fix Yourself Before It's Too Late", 'description': "Jordan B. Peterson, renowned psychologist and author, explains the importance of not wasting your life, how you should strive to be better every single day and how by fixing your life, you not only help yourself but everyone you come in contact with. \n\nSpecial Thanks to: \nDiary of a CEO: https://www.youtube.com/watch?v=3uLDin9A9pc \nMotivationHub: https://www.youtube.com/watch?v=S4vRiXumBoU&t=395s \nMotiversity: https://www.youtube.com/watch?v=cC_GsaiqS1U&t=318s \n\n►Listen to all our videos on the New Mindset App: https://bit.ly/M2SMindsetApp\n  ------------------------\nWays to stay connected with Motivation2Study and stay motivated all day lo

Category is 27. Now, let me find the category title

In [15]:


request_cat_id = youtube.videoCategories().list(
        part="snippet",
        id="27",
        access_token="snippet"
    )

response_cat_id = request_cat_id.execute()

category_name = response_cat_id["items"][0]["snippet"]["title"]


In [16]:
print(category_name)

Education


Lets pick 10 similar videos now as done in here: [CLICK](#section1) and query important information such as: title, descriptions, meta tags and comments 

Array of video links - for loop through them and split the id

In [44]:
video_list = ["https://www.youtube.com/watch?v=WEP5ubPMGDU", "https://www.youtube.com/watch?v=laSK7Pxh0_8", "https://www.youtube.com/watch?v=rhjiANJVR6g", "https://www.youtube.com/watch?v=wqEsTPaUZF0", "https://www.youtube.com/watch?v=dJyz6iK8VXE", "https://www.youtube.com/watch?v=4OmC6LyO5QI", "https://www.youtube.com/watch?v=-gYpCIbZjUQ", "https://www.youtube.com/watch?v=IKot8dQc1Ps","https://www.youtube.com/watch?v=yaXm6UsSHBM", "https://www.youtube.com/watch?v=FJV7HeHT4q4"]

new_list = []
for video in video_list:
    video_url = video.split("watch?v=")[-1]
    if "&t=" in video_url:
        video_id_new= video_url.split("&t=")[0]

    else:
        video_id_new = video_url

    new_list.append(video_id_new)

print(new_list)



['WEP5ubPMGDU', 'laSK7Pxh0_8', 'rhjiANJVR6g', 'wqEsTPaUZF0', 'dJyz6iK8VXE', '4OmC6LyO5QI', '-gYpCIbZjUQ', 'IKot8dQc1Ps', 'yaXm6UsSHBM', 'FJV7HeHT4q4']


Now that I have extracted all the Ids I can add them to the request

In [ ]:
import os
from collections import Counter
# import matplotlib.pyplot as plt
import pandas as pd
from dotenv import load_dotenv
from googleapiclient.discovery import build
from textblob import TextBlob

load_dotenv()

youtube_api_key = os.getenv("YOUTUBE_API_KEY")
youtube = build("youtube", "v3", developerKey=youtube_api_key)
request = youtube.videos().list(
    part="snippet,statistics",
    id=",".join(new_list)
)
api_response = request.execute()

# print(response)
# printed to test and is working :)!

data = []
comment_data = []

for index, item in enumerate(api_response.get("items", [])):  # Use enumerate() to get index
    data.append(
        {   
            "Index": index,
            "Title": item["snippet"]["title"],
            "Views": int(item["statistics"]["viewCount"]),
            "Likes": int(item["statistics"].get("likeCount", 0)),
            # "Comments": int(item["statistics"].get("commentCount", 0)),
            "CategoryId": item["snippet"]["categoryId"],
            "Published Date": item["snippet"]["publishedAt"],
            "Channel Name": item["snippet"]["channelTitle"],
            "Id": new_list[index]
        }
    )


data_df = pd.DataFrame(data)
print(data_df.head())

   Index                                              Title     Views   Likes  \
0      0  Jordan Peterson - How To Destroy Your Negative...  11120378  129480   
1      1  Jordan Peterson - 7 Harsh Truths To Take Contr...   5647156  121736   
2      2  DISCIPLINE YOURSELF -  Best Motivational Speec...   6076195  122956   
3      3  Jordan Peterson's Life Advice Will Change Your...  14851789  386693   
4      4  The Choice We All Have , But Only a Few Apply ...  10568618  226173   

  CategoryId        Published Date        Channel Name           Id  
0         22  2023-11-27T16:00:30Z    Chris Williamson  WEP5ubPMGDU  
1         22  2022-02-17T16:00:44Z    Chris Williamson  laSK7Pxh0_8  
2         27  2022-07-22T13:00:40Z   Chispa Motivation  rhjiANJVR6g  
3         27  2018-02-19T12:00:04Z  Motivation Madness  wqEsTPaUZF0  
4         27  2018-11-12T12:00:05Z  Motivation Madness  dJyz6iK8VXE  


In [40]:
data_df.to_csv("youtube_data.csv", index=False)


In [52]:
comment_data = []

for video_id in new_list:
    next_page_token = None  

    while True:  
        request_comments = youtube.commentThreads().list(
            part="snippet,replies",
            videoId=video_id,
            maxResults=100,  
            pageToken=next_page_token 
        )
        response_comments = request_comments.execute()

        
        for index, item in enumerate(response_comments.get("items", [])):
            comment = item["snippet"]["topLevelComment"]["snippet"]
            comment_data.append(
                {   
                    "Index": index,
                    "Comment": comment["textDisplay"],
                    "Comment Likes": comment["likeCount"],
                    "Reply Count": item["snippet"]["totalReplyCount"],
                    "Id": video_id  
                }
            )

        
        next_page_token = response_comments.get("nextPageToken")

        if not next_page_token:  
            break  

print(f"Total Comments Fetched: {len(comment_data)}")


Total Comments Fetched: 24492


In [53]:
comment_data_df = pd.DataFrame(comment_data)
print(comment_data_df.head())

   Index                                            Comment  Comment Likes  \
0      0  Hello you beauties. I really enjoyed this one,...           1220   
1      1                           Wonderful insight Jordan              0   
2      2  Jordan Peterson is a great man really genuine ...              0   
3      3  One of the greatest men of our time, God Bless...              1   
4      4  I’m not sure if it’s because of my ADHD, but a...              0   

   Reply Count           Id  
0           82  WEP5ubPMGDU  
1            0  WEP5ubPMGDU  
2            0  WEP5ubPMGDU  
3            0  WEP5ubPMGDU  
4            0  WEP5ubPMGDU  


In [54]:
comment_data_df.to_csv("youtube_comment_data.csv", index=False)

In [ ]:
description_tags_data = []

request_description_tags = youtube.videos().list(
   part="snippet,statistics",
    id=",".join(new_list)
)
response_description_tags = request_description_tags.execute()

for index, item in enumerate(response_description_tags.get("items", [])):  # Use enumerate() to get index
    description_tags_data.append(
        {
            "Index": index,
            "Tags": item["snippet"]["tags"],
            "Description": item["snippet"]["description"]
        }
    )

description_tags_df = pd.DataFrame(description_tags_data)


In [59]:
print(description_tags_df.head())

   Index                                               Tags  \
0      0  [modern wisdom, podcast, chris williamson, Chr...   
1      1  [modern wisdom, podcast, chris williamson, Jor...   
2      2  [jordan peterson, jordan peterson motivation, ...   
3      3  [motivational video, motivation for 2018, moti...   
4      4  [motivational video, motivation for 2018, moti...   

                                         Description  
0  Dr Jordan B. Peterson is a clinical psychologi...  
1  Dr. Jordan B. Peterson is Professor Emeritus a...  
2  Subscribe for weekly videos.\n\nFollow us on S...  
3  Pre-Order Jordan Peterson's #1 NEW RELEASE Boo...  
4  Jordan Peterson's Life Advice Will Change Your...  


In [60]:
description_tags_df.to_csv("youtube_description_and_tags.csv", index=False)